In [3]:
import jsonc

In [4]:
original = "loottables_original.json"

In [5]:
with open(original, "r") as file:
    loottable_dict = jsonc.load(file)

In [6]:
for key in loottable_dict:
    print(key)

MagicEffectsCount
RestrictedItems
ItemSets
LootTables


## MagicEffectsCount Functions
### Functions that change the ammount, and it's associated probability, of enchantments

In [7]:
def increase_enchant_count(n):
    """
    Increases all magic effects count by n.
    Changes probabilities only indirectly.
    """
    print("########################## BEFORE ###########################")
    for rarity, probabilities in loottable_dict["MagicEffectsCount"].items():
        print(f"\t  {rarity}: \t{probabilities}")
        for i in range(len(probabilities)):
            probabilities[i][0] += 1
    
    print("########################## AFTER ###########################")
    for rarity, probabilities in loottable_dict["MagicEffectsCount"].items():
        print(f"\t  {rarity}: \t{probabilities}")
    
    print("############################################################")
    print("##################### INCREASE COUNT #######################")
    print("############################################################")
    print()
    print()
    return 1

def modify_probabilities(p):
    """
    Takes p away from the first entry and adds 2*p/3 and p/3 to the second and last entry, respectively
    """
    print("########################## BEFORE ###########################")
    for rarity, probabilities in loottable_dict["MagicEffectsCount"].items():
        print(f"\t  {rarity}: \t{probabilities}")
        probabilities[0][1] -= p
        probabilities[1][1] += (p//3) * 2
        probabilities[2][1] = 100 - probabilities[0][1] - probabilities[1][1]
    
    print("########################## AFTER ###########################")
    for rarity, probabilities in loottable_dict["MagicEffectsCount"].items():
        print(f"\t  {rarity}: \t{probabilities}")
    
    print("############################################################")
    print("################### MODIFY PROBABILITIES ###################")
    print("############################################################")
    print()
    print()
    return 1

## ItemSets Functions
### Functions that create/delete item sets

###### Note: EpicLoot provides a setting to determine the item-to-material drop ratio. The only provided function to change item sets in this code aims to improve configuration in case you are running CLLC and want only materials dropping. In such cases, utilising solely the EpicLoot configuration, I was met with bosses trophies drop not scaling with starts. The solution was to manually change the loottables.json file.

In [8]:
def only_enchanting_mats():
    """
    Deletes all item sets that don't include enchanting mats
    """
    item_set_index = 0
    while item_set_index < len(loottable_dict["ItemSets"]):
        item_set = loottable_dict["ItemSets"][item_set_index]
        #print(item_set["Name"], [char_seq in item_set["Name"] for char_seq in ["Mats", "Runestone"]])
        if not any([char_seq in item_set["Name"] for char_seq in ["Mats", "Runestone"]]):
            print(item_set["Name"], "Deleted")
            loottable_dict["ItemSets"].pop(item_set_index)
        else:
            item_set_index += 1

## Non-vanilla mobs

In [9]:
def add_mob_objects():
    """
    Creates mob objects for all mob ids added in the mobs_additions.txt file, each with their
    respective tier, also defined in the file
    """
    with open("mobs_additions.txt", "r") as mobs_file:
        for line in mobs_file:
            if line[0] == "#" or line == "\n":
                continue
            mob_id, tier = line.replace("\n", "").split()
            print(f"{mob_id}{(20-len(mob_id)) * ' '}\t{tier}", end=" ")
            mob_obj = {}
            mob_obj["Object"] = mob_id
            mob_obj["RefObject"] = f"Tier{tier}Mob"
            loottable_dict["LootTables"].insert(8, mob_obj)
            print(mob_obj, f"{(65-len(str(mob_obj))) * ' '}ADDED")

## Generate Mob Loot Tables
### Functions to generate the loot tables for mob tiers and bosses

In [97]:
for b in loottable_dict["LootTables"][:1]:
    print(b)

{'Object': 'Tier0Mob', 'LeveledLoot': [{'Level': 1, 'Drops': [[0, 100]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [100, 0, 0, 0]}]}, {'Level': 2, 'Drops': [[0, 95], [1, 5]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [95, 5, 0, 0]}]}, {'Level': 3, 'Drops': [[0, 90], [1, 10]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [90, 10, 0, 0]}]}, {'Level': 4, 'Drops': [[0, 80], [1, 20]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [80, 15, 5, 0]}]}, {'Level': 5, 'Drops': [[0, 70], [1, 30]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [70, 19, 10, 1]}]}, {'Level': 6, 'Drops': [[0, 60], [1, 39], [2, 1]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [60, 20, 15, 5]}]}, {'Level': 7, 'Drops': [[0, 60], [1, 39], [2, 1]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [60, 20, 15, 5]}]}, {'Level': 8, 'Drops': [[0, 60], [1, 39], [2, 1]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rari

In [10]:
tiers = set()
for obj in loottable_dict["LootTables"]:
    if "RefObject" in obj.keys():
        tiers.add(obj["RefObject"])
tiers = list(tiers)
tiers.sort()
print(tiers)

['Tier0Mob', 'Tier1Mob', 'Tier2Mob', 'Tier3Mob', 'Tier4Mob', 'Tier5Mob', 'Tier6Mob', 'Tier7Mob']


In [285]:
import numpy as np

def aux_calc(level, tier):
    if tier == 0:
        tier += 1
    return int(np.log2(tier)*1.2) + int(np.log2(level)*1.4) + 1
    
def generate_mobs_loot_tables(max_star):
    """
    Generate the loot tables for each mob tier based on the maximum starts fro mobs
    """
    for tier in tiers:
        tier_num = int(tier[4])
        print(f"TIER {tier}")
        leveled_loot = []
        for level in range(1, max_star+1):
            print("level", level, end=": ")
            level_loot_obj = {}
            level_drops = []
            for drop_count in range(max(0, aux_calc(level, tier_num)-2), aux_calc(level, tier_num)+1):      
                print(f"{drop_count}", end=" ")
            print()

In [286]:
generate_mobs_loot_tables(8)

TIER Tier0Mob
level 1: 0 1 
level 2: 0 1 2 
level 3: 1 2 3 
level 4: 1 2 3 
level 5: 2 3 4 
level 6: 2 3 4 
level 7: 2 3 4 
level 8: 3 4 5 
TIER Tier1Mob
level 1: 0 1 
level 2: 0 1 2 
level 3: 1 2 3 
level 4: 1 2 3 
level 5: 2 3 4 
level 6: 2 3 4 
level 7: 2 3 4 
level 8: 3 4 5 
TIER Tier2Mob
level 1: 0 1 2 
level 2: 1 2 3 
level 3: 2 3 4 
level 4: 2 3 4 
level 5: 3 4 5 
level 6: 3 4 5 
level 7: 3 4 5 
level 8: 4 5 6 
TIER Tier3Mob
level 1: 0 1 2 
level 2: 1 2 3 
level 3: 2 3 4 
level 4: 2 3 4 
level 5: 3 4 5 
level 6: 3 4 5 
level 7: 3 4 5 
level 8: 4 5 6 
TIER Tier4Mob
level 1: 1 2 3 
level 2: 2 3 4 
level 3: 3 4 5 
level 4: 3 4 5 
level 5: 4 5 6 
level 6: 4 5 6 
level 7: 4 5 6 
level 8: 5 6 7 
TIER Tier5Mob
level 1: 1 2 3 
level 2: 2 3 4 
level 3: 3 4 5 
level 4: 3 4 5 
level 5: 4 5 6 
level 6: 4 5 6 
level 7: 4 5 6 
level 8: 5 6 7 
TIER Tier6Mob
level 1: 2 3 4 
level 2: 3 4 5 
level 3: 4 5 6 
level 4: 4 5 6 
level 5: 5 6 7 
level 6: 5 6 7 
level 7: 5 6 7 
level 8: 6 7 8 
TIER Tier7

In [216]:
for i in range(1, 9):
    print(f"level {i}:", end=" ")
    if i == 1:
        a = abs(np.log2(i))
        print(int(a))
    else:
        a = abs(0.9 - (np.log2(i) - np.log2(i-1)))
        #print(a, end=" ")
        a *= 6
        print(round(a))

level 1: 0
level 2: 1
level 3: 2
level 4: 3
level 5: 3
level 6: 4
level 7: 4
level 8: 4


In [106]:
for lotable in loottable_dict["LootTables"][:1]:
    for level in lotable["LeveledLoot"]:
        print(level)

{'Level': 1, 'Drops': [[0, 100]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [100, 0, 0, 0]}]}
{'Level': 2, 'Drops': [[0, 95], [1, 5]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [95, 5, 0, 0]}]}
{'Level': 3, 'Drops': [[0, 90], [1, 10]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [90, 10, 0, 0]}]}
{'Level': 4, 'Drops': [[0, 80], [1, 20]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [80, 15, 5, 0]}]}
{'Level': 5, 'Drops': [[0, 70], [1, 30]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [70, 19, 10, 1]}]}
{'Level': 6, 'Drops': [[0, 60], [1, 39], [2, 1]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [60, 20, 15, 5]}]}
{'Level': 7, 'Drops': [[0, 60], [1, 39], [2, 1]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [60, 20, 15, 5]}]}
{'Level': 8, 'Drops': [[0, 60], [1, 39], [2, 1]], 'Loot': [{'Item': 'Tier0Everything', 'Weight': 1, 'Rarity': [60, 20, 15, 5]}]}
